In [7]:
import mlflow
import pandas as pd

# Set the tracking URI to your DagsHub MLflow instance
mlflow.set_tracking_uri("https://dagshub.com/HassanBarka/MLOps.mlflow") 
reports_path = "./models/run_info.json"
with open(reports_path, 'r') as file:
    run_info = json.load(file)

model_name = run_info['model_name']  



try:
    # Create an MlflowClient to interact with the MLflow server
    client = mlflow.tracking.MlflowClient()

    # Get the latest version of the model in the Production stage
    versions = client.get_latest_versions(model_name, stages=["Production"])

    if versions:
        latest_version = versions[0].version
        run_id = versions[0].run_id  # Fetching the run ID from the latest version
        print(f"Latest version in Production: {latest_version}, Run ID: {run_id}")

        # Construct the logged_model string
        logged_model = f'runs:/{run_id}/{model_name}'
        print("Logged Model:", logged_model)

        # Load the model using the logged_model variable
        loaded_model = mlflow.pyfunc.load_model(logged_model)
        print(f"Model loaded from {logged_model}")

        # # Input data for prediction
        # data = pd.DataFrame({
        #     'ph': [3.71608],
        #     'Hardness': [204.89045],
        #     'Solids': [20791.318981],
        #     'Chloramines': [7.300212],
        #     'Sulfate': [368.516441],
        #     'Conductivity': [564.308654],
        #     'Organic_carbon': [10.379783],
        #     'Trihalomethanes': [86.99097],
        #     'Turbidity': [2.963135]
        # })

        # # Make prediction
        # prediction = loaded_model.predict(data)
        # print("Prediction:", prediction)
    else:
        print("No model found in the 'Production' stage.")

except Exception as e:
    print(f"Error fetching model: {e}")

/tmp/ipykernel_58832/824515244.py:19: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(model_name, stages=["Production"])


Latest version in Production: 4, Run ID: 6c6a46cd33d14bf2aef430592b04cdd3
Logged Model: runs:/6c6a46cd33d14bf2aef430592b04cdd3/DT
Model loaded from runs:/6c6a46cd33d14bf2aef430592b04cdd3/DT


In [21]:
from backend.example_json.drought_info import DroughtModel
from backend.preprocessing import scale_data,date_processing

In [12]:
example_data = DroughtModel.Config.json_schema_extra["example"]
drought_instance = DroughtModel(**example_data)
drought_instance

DroughtModel(fips=1001, date=datetime.datetime(2000, 1, 4, 0, 0), PRECTOT=15.95, PS=100.29, QV2M=6.42, T2M=11.4, T2MDEW=6.09, T2MWET=6.1, T2M_MAX=18.09, T2M_MIN=2.16, T2M_RANGE=15.92, TS=11.31, WS10M=3.84, WS10M_MAX=5.67, WS10M_MIN=2.08, WS10M_RANGE=3.59, WS50M=6.73, WS50M_MAX=9.31, WS50M_MIN=3.74, WS50M_RANGE=5.58)

In [58]:
data= pd.DataFrame(drought_instance.dict(),index=[0])
data

/tmp/ipykernel_58832/761339514.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data= pd.DataFrame(drought_instance.dict(),index=[0])


,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,1001,2000-01-04,15.95,100.29,6.42,11.4,6.09,6.1,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58


In [59]:
import pickle
from backend.preprocessing import scale_data,date_processing
data = date_processing(data)

df = scale_data(data)
df

TypeError: file must have 'read' and 'readline' attributes

In [43]:
data

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,year,month,day
0,1001,15.95,100.29,6.42,11.4,6.09,6.1,18.09,2.16,15.92,...,5.67,2.08,3.59,6.73,9.31,3.74,5.58,2000,1,4


In [56]:
# Colonnes attendues par le modèle
columns = ['fips', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
           'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
           'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
           'WS50M_RANGE', 'year', 'month', 'day']

# Conversion en DataFrame
df = pd.DataFrame(df, columns=columns)
loaded_model.predict(df)

array([1])